# Phase 2 — Supervised Learning
In this phase, we build and evaluate supervised machine learning models to predict the disease (prognosis) from symptom-based features (0/1). We compare two models: **Decision Tree** and **Naïve Bayes**.


## 1) Algorithm Selection & Justification

**Selected models:**
1. **Decision Tree Classifier**  
   - Works very well with binary features (0/1 symptoms).  
   - Can capture non-linear relations between symptoms and diseases.

2. **Naïve Bayes (BernoulliNB)**  
   - Very fast and simple.  
   - Designed for binary features → perfect for our dataset (symptom present = 1, absent = 0).  
   - Useful as a baseline model to compare with tree-based models.


In [10]:
import pandas as pd
from pathlib import Path

# Define file paths for the training and testing datasets
train_path = Path("Training.csv") 
test_path = Path("Testing.csv") 

# Load the datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# emove any unnecessary columns (e.g., "Unnamed" columns automatically generated by CSV export) (A precautionary step)
train_df = train_df.loc[:, ~train_df.columns.str.contains('^Unnamed')]
test_df = test_df.loc[:, ~test_df.columns.str.contains('^Unnamed')]

# Display dataset shapes to confirm successful loading
print("Training data:", train_df.shape)
print("Testing data:", test_df.shape)

# Identify the feature columns (symptoms) and target variable (disease prognosis)
symptom_cols = [c for c in train_df.columns if c != 'prognosis']

#Split the data into input features (X) and target output (y)
X_train = train_df[symptom_cols].astype(int)
y_train = train_df['prognosis']

X_test = test_df[symptom_cols].astype(int)
y_test = test_df['prognosis']

#Print summary information about features and target
print("Number of features:", len(symptom_cols))
print("Unique classes (diseases):", y_train.nunique())


Training data: (4920, 133)
Testing data: (42, 133)
Number of features: 132
Unique classes (diseases): 41


In [24]:
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Initialize a Decision Tree model
# Using Gini impurity to measure node splits and limiting depth to avoid overfitting
dt = DecisionTreeClassifier(
    criterion="gini",
    max_depth=32,
    random_state=42
)

#Train (fit) the model on the training dataset
dt.fit(X_train, y_train)

# Predict the disease (prognosis) for the test set
y_pred_dt = dt.predict(X_test)

# Evaluate model performance
print("=== Decision Tree Results ===")
print("Accuracy:", round(accuracy_score(y_test, y_pred_dt), 4))
print("\nClassification Report:\n",
      classification_report(y_test, y_pred_dt, zero_division=0))

# Identify the most influential symptoms for decision-making
importances = pd.Series(dt.feature_importances_, index=symptom_cols)
print("\nTop 10 important symptoms (Decision Tree):\n")
print(importances.sort_values(ascending=False).head(10))



=== Decision Tree Results ===
Accuracy: 0.881

Classification Report:
                                          precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       0.20      1.00      0.33         1
                                   Acne       0.00      0.00      0.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       0.00      0.00      0.00         1
                            Common Co

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Initialize a Random Forest model
# Using 100 trees and random_state for reproducibility
rf = RandomForestClassifier(
    n_estimators=100,
    criterion="gini",
    random_state=42,
    n_jobs=-1
)

# Train (fit) the model on the training dataset
rf.fit(X_train, y_train)

# Predict the disease (prognosis) for the test set
y_pred_rf = rf.predict(X_test)

# Evaluate model performance
print("=== Random Forest Results ===")
print("Accuracy:", round(accuracy_score(y_test, y_pred_rf), 4))
print("\nClassification Report:\n",
      classification_report(y_test, y_pred_rf, zero_division=0))

# Identify the most influential symptoms for decision-making
importances = pd.Series(rf.feature_importances_, index=symptom_cols)
print("\nTop 10 important symptoms (Random Forest):\n")
print(importances.sort_values(ascending=False).head(10))

# (Optional) Since Random Forest has many trees, we show feature importance instead of individual decision rules
print("\nNote: Random Forest consists of multiple Decision Trees.")
print("Below are the top symptoms contributing to predictions.")


=== Random Forest Results ===
Accuracy: 0.9762

Classification Report:
                                          precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       1.00      1.00      1.00         1
                            Common C

In [25]:
from sklearn.naive_bayes import BernoulliNB


# Initialize the Naïve Bayes model (BernoulliNB)
# BernoulliNB is well-suited for binary feature datasets (0/1), such as symptoms being present or absent
nb = BernoulliNB(alpha=1.0) # alpha = 1.0 applies Laplace smoothing to handle zero-frequency issues


# Train (fit) the model on the training dataset
nb.fit(X_train, y_train)

# Predict disease prognosis for the test dataset
y_pred_nb = nb.predict(X_test)

#  Evaluate model performance
print("=== Naïve Bayes (Bernoulli) Results ===")
print("Accuracy:", round(accuracy_score(y_test, y_pred_nb), 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred_nb))


# Note: The dataset is highly structured and well-separated.
# Each disease has a unique combination of binary symptoms (0/1),
# which makes it easy for Naïve Bayes to classify perfectly.
# Therefore, the 100% accuracy here does NOT indicate overfitting,
# but rather that the dataset is very clean and clearly separable.



=== Naïve Bayes (Bernoulli) Results ===
Accuracy: 1.0

Classification Report:
                                          precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       1.00      1.00      1.00         1
                            C

In [26]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Calculate the accuracy for each model
dt_acc = accuracy_score(y_test, y_pred_dt)
nb_acc = accuracy_score(y_test, y_pred_nb)
rf_acc = accuracy_score(y_test, y_pred_rf)

# Create a DataFrame to compare model performance
results_df = pd.DataFrame({
    'Model': ['Decision Tree', 'Naïve Bayes (Bernoulli)', 'Random Forest'],
    'Accuracy': [dt_acc, nb_acc, rf_acc]
})

# Convert accuracy values to percentages (rounded to 2 decimal places)
results_df['Accuracy'] = (results_df['Accuracy'] * 100).round(2).astype(str) + '%'

# Display the final comparison table
print("=== Model Accuracy Comparison ===")
display(results_df)


=== Model Accuracy Comparison ===


Model Accuracy
0            Decision Tree    88.1%
1  Naïve Bayes (Bernoulli)   100.0%
2            Random Forest   97.62%

In [27]:
from sklearn.model_selection import cross_val_score

# Perform 5-Fold Cross-Validation on the training dataset
# This technique evaluates how well each model generalizes
# by training and testing on different splits within the training data.

# Using the training portion only
X_all = X_train
y_all = y_train

# Decision Tree Cross-Validation
dt_cv = cross_val_score(dt, X_all, y_all, cv=5).mean()

# Naïve Bayes Cross-Validation
nb_cv = cross_val_score(nb, X_all, y_all, cv=5).mean()

# Random Forest Cross-Validation
rf_cv = cross_val_score(rf, X_all, y_all, cv=5).mean()

# Display mean accuracy scores across the 5 folds
print("=== 5-Fold Cross-Validation Results ===")
print("Decision Tree:", round(dt_cv, 4))
print("Naïve Bayes (Bernoulli):", round(nb_cv, 4))
print("Random Forest:", round(rf_cv, 4))



=== 5-Fold Cross-Validation Results ===
Decision Tree: 0.8827
Naïve Bayes (Bernoulli): 1.0
Random Forest: 1.0


## Results Interpretation

After training and evaluating all three models on the symptom–disease dataset, the following observations were made:

- **Decision Tree (max_depth = 6)** achieved an accuracy of approximately **0.88**.  
  The low performance is due to the large number of binary features (over 130) and multiple output classes (42 diseases).  
  A shallow tree cannot represent all possible decision paths, resulting in *underfitting*.  
  When the depth restriction is removed, the accuracy improves but increases the risk of *overfitting*.

- **Naïve Bayes (BernoulliNB)** achieved a perfect **1.00 (100%) accuracy**.  
  This is because each disease in the dataset has a very distinct combination of binary symptoms, which aligns perfectly with Naïve Bayes’ probabilistic independence assumption.  
  However, such perfect accuracy suggests that the dataset is highly separable and may not reflect real-world variability.

- **Random Forest (n_estimators = 100)** achieved an accuracy of around **0.97**, which is both high and realistic.  
  By combining multiple decision trees, the model reduces overfitting and generalizes better compared to a single tree.  
  It balances interpretability and predictive power, making it the most reliable among the three.

---

### Comparison

| Model | Accuracy | Strengths | Weaknesses |
|--------|-----------|------------|-------------|
| **Decision Tree** | ~0.88 | Interpretable, easy to visualize | Underfits when depth is limited |
| **Naïve Bayes (Bernoulli)** | 1.00 | Excellent for binary, separable data | May overfit on clean datasets |
| **Random Forest** | ~0.97 | High accuracy, robust to overfitting | Less interpretable than a single tree |

---

### Conclusion

Among the three models, **Random Forest** provided the most balanced performance achieving high accuracy without perfect overfitting.  
**Naïve Bayes** performed exceptionally well because of the binary and highly structured nature of the dataset, but it might not generalize well in noisier, real-world data.  
**Decision Tree**, although less accurate, remains valuable for explaining model decisions and understanding which symptoms are most influential.

In a real-world healthcare decision support system, combining **Random Forest** for robust prediction and **Decision Tree** for interpretability would deliver both **accuracy** and **transparency**, which are critical in medical AI applications.
